In [1]:
import openai
import os
import pandas as pd	


# provide api key (accessed here from Environment Variables)
openai.api_key = os.getenv("OPENAI_API_KEY")

# inspection and overview of openai available models


In [5]:
# get list of models
model_list = openai.Model.list()
# convert json to dataframe
df = pd.json_normalize(model_list.data)	
df.to_excel("../model_list.xlsx")
df

,id,object,created,owned_by,permission,root,parent
0,babbage,model,1649358449,openai,"[{'id': 'modelperm-49FUp5v084tBB49tC4z8LPH5', ...",babbage,None
1,davinci,model,1649359874,openai,"[{'id': 'modelperm-U6ZwlyAd0LyMk4rcMdz33Yc3', ...",davinci,None
2,gpt-3.5-turbo-0301,model,1677649963,openai,"[{'id': 'modelperm-ms2DpLH5OCOZpUikJ4sRQNkh', ...",gpt-3.5-turbo-0301,None
3,text-davinci-003,model,1669599635,openai-internal,"[{'id': 'modelperm-6CAfTW5IbFpnlziQKoDilahq', ...",text-davinci-003,None
4,babbage-code-search-code,model,1651172509,openai-dev,"[{'id': 'modelperm-4qRnA3Hj8HIJbgo0cGbcmErn', ...",babbage-code-search-code,None
...,...,...,...,...,...,...,...
64,davinci-instruct-beta:2.0.0,model,1629501914,openai,"[{'id': 'snapperm-c70U4TBfiOD839xptP5pJzyc', '...",davinci-instruct-beta:2.0.0,None
65,text-ada:001,model,1641949608,system,"[{'id': 'snapperm-d2PSnwFG1Yn9of6PvrrhkBcU', '...",text-ada:001,None
66,text-davinci:001,model,1641943966,system,"[{'id': 'snapperm-Fj1O3zkKXOQy6AkcfQXRKcWA', '...",text-davinci:001,None
67,text-curie:001,model,1641955047,system,"[{'id': 'snapperm-BI9TAT6SCj43JRsUb9CYadsz', '...",text-curie:001,None


## Conversation (GPT3.x)

### initialize model

The model posesses two different roles:
1. system
2. user 

**set context and behavior of the system**<br>
as first message to system

In [1]:
# initialization outside of chat loop to keep order of queries
messages = [
    # system message first, it helps set the behavior of the assistant (default)
    {"role": "system", 
     "content": "You are a kind, helpful assistant."}, # could also be sarcastic troll
]

### chat
**start (infinite) loop for user input**

In [3]:
while True:
    message = input("👨‍💻: ")
    # define user role and process input
    if message:
        messages.append(
            {"role": "user", "content": message},
            )
    # define assistant model and process output
    chat = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", # "gpt-3.5-turbo" same as in current chatgpt app 
        messages=messages,
        max_tokens=1024, # limit max. token spending
        temperature=1,   # randomness of assistant. default = 1 
        )
    # extract first response [0]
    reply = chat.choices[0].message.content
    # print assistant response
    print(f"🤖: {reply}")
    messages.append(
        {"role": "assistant", "content": reply}
        )
    

In [3]:
input("Press Enter to continue...")

## Image generation (Dall-E)

### from scratch

In [11]:
prompt_for_image = "3d render of a cute robotic kraken with six arms. Microphone in one arm paintbrush in another arm, sketchbook in another arm, computer in another arm"
#"3d render of a cute octopus painting, writing reading, coding and listening at the same time"
number_of_images = 1 # 1-10 can be requested at a time
image_size = "1024x1024" # 256x256, 512x512, 1024x1024, 2048x2048


# execute
response = openai.Image.create(
  prompt=prompt_for_image,
  n=number_of_images,
  size=image_size
)

# return as url (alternatively as base64)
image_url = response['data'][0]['url']

In [ ]:
image_path = "../dalle_images/"

In [20]:
import requests

image_path = "../dalle_images/"
# image name without whitespaces and points and max 20 characters
image_name = image_path + prompt_for_image.replace(" ", "_").replace(".", "")[:35]

img_data = requests.get(image_url).content
with open(f'{image_name}.jpg', 'wb') as handler:
    handler.write(img_data)

### edit and variation of existing image 


In [ ]:
POST https://api.openai.com/v1/images/edits
POST https://api.openai.com/v1/images/variations

## speech input (Whisper)

- transcribe
- translate

In [ ]:
# record speech from pc-microphone


### transcribe

In [ ]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
audio_file= open("/path/to/file/audio.mp3", "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)

### translate

-----

# File handling


### overview of files

In [ ]:
!curl https://api.openai.com/v1/files -H 'Authorization: Bearer OPENAI_API_KEY'